In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
random_seed = 42

In [2]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True)

In [3]:
X_train

,lobby_type,r1_xp,r1_max_health,r1_max_mana,r1_x,r1_y,r2_xp,r2_max_health,r2_max_mana,r2_x,...,d4_xp,d4_max_health,d4_max_mana,d4_x,d4_y,d5_xp,d5_max_health,d5_max_mana,d5_x,d5_y
492,0.0,11473.0,1380.0,1094.939,116.0,160.0,8599.0,1435.0,945.938,72.0,...,14946.0,2000.0,830.938,182.0,176.0,10459.0,2150.0,614.938,180.0,174.0
10529,7.0,1020.0,680.0,362.938,84.0,168.0,1575.0,720.0,506.938,170.0,...,1400.0,860.0,374.938,174.0,86.0,1738.0,880.0,386.938,130.0,130.0
366,7.0,12597.0,2160.0,518.938,114.0,110.0,8801.0,1480.0,1106.939,98.0,...,17722.0,2045.0,1327.939,158.0,94.0,12786.0,1955.0,1279.939,140.0,122.0
3845,7.0,26930.0,2460.0,1104.939,92.0,134.0,24383.0,1760.0,1214.939,106.0,...,16036.0,1880.0,1350.939,128.0,158.0,27211.0,2560.0,926.938,110.0,108.0
18722,7.0,2401.0,920.0,386.938,178.0,102.0,1340.0,860.0,518.938,168.0,...,2641.0,900.0,434.938,152.0,130.0,2315.0,840.0,314.938,160.0,126.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22255,0.0,9296.0,1120.0,1092.939,74.0,78.0,20727.0,1950.0,818.938,86.0,...,21714.0,2655.0,1747.939,180.0,174.0,17357.0,1635.0,1437.939,184.0,174.0
5549,7.0,20831.0,2260.0,662.938,102.0,158.0,21627.0,2045.0,2116.939,104.0,...,13838.0,1795.0,1147.939,120.0,138.0,21639.0,1760.0,974.938,124.0,130.0
876,7.0,27851.0,2500.0,1586.939,176.0,166.0,26992.0,2180.0,758.938,168.0,...,26911.0,2020.0,650.938,130.0,144.0,18907.0,1795.0,1713.939,156.0,154.0
16284,0.0,4036.0,1415.0,657.938,92.0,138.0,4236.0,760.0,530.938,78.0,...,8722.0,1760.0,746.938,148.0,128.0,4980.0,1280.0,458.938,128.0,126.0


In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
model = SVC(kernel='rbf', gamma= 10, C=10)
model.fit(X_train, Y_train)

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=10, gamma=10)

In [4]:
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict(X_test))}")

Score : 0.5317371937639198
ROC-AUC score : 0.5


In [5]:
ut.write(model)